In [1]:
##data load ### cpu 버전

import os
import pickle
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
from data_cpu import Dataset, LMOrderedIterator
from model import TFTransfoXLModel,TFTransfoXLLMHeadModel

from transformers import TransfoXLConfig


config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/wiki_short/',
    dataset = 'wt103',
    d_embed=128,
    d_head = 32,
    d_model=64,
    mem_len=400,
    n_head=8,
    n_layer=6,
    batch_size = 10,
    tgt_len = 26,
    ext_len = 0,
    eval_tgt_len = 36
)



kwargs = {}
if config_xl.dataset in ['wt103', 'wt2']:
    kwargs['special'] = ['<eos>','<UNK>']
    kwargs['lower_case'] = False

dataset = Dataset(**kwargs)

train_dataset, val_dataset, test_dataset = dataset.make_dataset(config_xl.data,config_xl.dataset)

# strategy = tf.distribute.MirroredStrategy()

2023-11-07 17:08:17.845686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 17:08:18.396634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-07 17:08:18.396676: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-07 17:08:18.396681: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Producing dataset wt103...
final vocab size 109697 from 109695 unique tokens


In [2]:
400 // 10*1

40

분산처리를 하기위해서는 train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
이 코드를 실행해야하는데ㅐ 기본 텐서 구조는 안되더라 그래서 텐서 슬라이스 변경해서 이 구조로 바꾸고 데이터를 이터레이션 해서 모델에 집어 넣어야 하는 방식을 고민중 


In [3]:
def gen(data,bsz,bptt,ext_len=None,):
  
  bsz = bsz#3 #60
  bptt = bptt#36 #70
  ext_len = ext_len if ext_len is not None else 0
  data = data
  
  
  # Work out how cleanly we can divide the dataset into bsz parts.
  # 아래의 두 코드는   data 텐서에서 배치 크기 bsz로 깔끔하게 맞지 않는 추가 요소를 제거하는 것 배치에 띡 떨어지게
  n_step = len(data) // (bsz*bptt)
  print('n_step',n_step) # 312779
  
  sliced_data = tf.slice(data,[0],[n_step * bsz*bptt])  
  print('sliced_data',sliced_data)
  # sliced_data = self.data[:self.n_step * self.bsz]
  '''# 시작 위치와 슬라이싱할 크기 설정
  begin = [0]  # 첫 번째 차원의 시작 위치는 0
  size = [6]   # 첫 번째 차원에서 6개의 원소를 슬라이싱

  # 데이터를 잘라내기 (tf.slice 사용)
  sliced_data = tf.slice(data, begin, size)  '''

  # Evenly divide the da
  # ta across the bsz batches.


  new_shape = (bsz, -1)  # 나머지 차원은 자동으로 계산됨
  data_reshaped = tf.reshape(sliced_data, new_shape)
  # data_transposed = tf.transpose(data_reshaped)
  data = data_reshaped
  # print('data',len(data))

  n_batch = (n_step + bptt - 1) // bptt

  for i in range(0, len(data[1]) - 1, bptt):
    
    seq_len = min(bptt, data.shape[1] - 1 - i) # # i값이 103227020를 넘지 않는 이상 seq_len = 70


    end_idx = i + seq_len # 70,71,72,73,74......
    beg_idx = max(0, i - ext_len) # 0,1,2,3,4,5
    ''' 아래 처럼 첫번째 차원을 자르는 이류
    로,또,1,등,당,첨 = > 로,또,1    => 로, 등
                    등,당,첨         또, 당
                                    1, 첨
    '''
    p_data = data[:,beg_idx:end_idx] # self.data[:,0:70],[:,1:71] ~
    target = data[:,i+1:i+1+seq_len]
    if i + bptt < len(data[1]) - 1:
      yield p_data, target


In [4]:
dataset = tf.data.Dataset.from_generator(
     gen,
     output_signature=(
         tf.TensorSpec(shape=None, dtype=tf.int32),
         tf.TensorSpec(shape=None, dtype=tf.int32),
         ),
     args=(train_dataset,config_xl.batch_size,config_xl.tgt_len)
         )

2023-11-07 17:08:24.049059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 17:08:24.049442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 17:08:24.054995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 17:08:24.055352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-07 17:08:24.055691: I tensorflow/compiler/xla/stream_executo

In [5]:
sample=next(iter(dataset))
data, target = sample
print(target.shape)

n_step 21653
sliced_data tf.Tensor([   0   10 8986 ...   28 1502 1423], shape=(5629780,), dtype=int32)
(10, 26)


In [6]:
# strategy = tf.distribute.MirroredStrategy()
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# BATCH_SIZE_PER_REPLICA = 1
# GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 2

In [7]:
# train_dist_dataset = strategy.experimental_distribute_dataset(dataset)


In [8]:
# batched_dataset =dataset.batch(GLOBAL_BATCH_SIZE)
dataset.cache().prefetch(tf.data.AUTOTUNE)

<PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [9]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = tf.cast(d_model, tf.float32)

    self.warmup_steps = tf.cast(warmup_steps,tf.float32)

  def __call__(self, step):
    step =tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [10]:
learning_rate = CustomSchedule(config_xl.d_model)


In [11]:
# with strategy.scope():
#   def compute_loss(loss):
    
#     return tf.nn.compute_average_loss(loss, global_batch_size=GLOBAL_BATCH_SIZE)

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [13]:
# with strategy.scope():

model = TFTransfoXLLMHeadModel(config=config_xl)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                    epsilon=1e-9)
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=model,
                        optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')

In [14]:
def train_step(data, target,mems):
  

  with tf.GradientTape() as tape:
    outputs = model(input_ids=data,labels=target,mems=mems)
    loss = outputs.loss
    mems = outputs.mems

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  return mems,loss

In [15]:
for epoch in range(6):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  mems = None
  for batch, (data, target) in enumerate(dataset):
    
    mems,loss = train_step(data, target,mems)
    
    
    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

n_step 21653
sliced_data tf.Tensor([   0   10 8986 ...   28 1502 1423], shape=(5629780,), dtype=int32)


2023-11-07 17:08:26.070814: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55a387bc6750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-07 17:08:26.070833: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-11-07 17:08:26.070836: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-11-07 17:08:26.074086: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-07 17:08:26.150222: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 Batch 0 Loss 10.3165 Accuracy 0.0000
Epoch 1 Batch 50 Loss 10.4666 Accuracy 0.0000
Epoch 1 Batch 100 Loss 10.5108 Accuracy 0.0000


2023-11-07 17:09:03.841479: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at strided_slice_op.cc:105 : INVALID_ARGUMENT: Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1], [26,10], and [1] instead.
2023-11-07 17:09:03.843622: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1], [26,10], and [1] instead. [Op:StridedSlice] name: strided_slice/
Traceback (most recent call last):

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1], [26,10], and [1] instead. [Op:StridedSlice] name: strided_slice/
Traceback (most recent call last):

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/jun/miniconda3/envs/new/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1039, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_500576/4167695319.py", line 50, in g

여기서 cache   prefetch 등 사용해서 gpu till 100%로 끓어올려보자

In [ ]:
# # 결과 텐서의 형태를 계산합니다.
# result_shape = (q.shape[0], k.shape[0], q.shape[1], q.shape[2])

# # 결과 텐서를 저장할 텐서를 생성합니다.
# result = tf.Variable(tf.zeros(result_shape, dtype=tf.int32))

# # 모든 i, j, b, n에 대해 연산을 수행합니다.
# for i in range(result_shape[0]):
#     for j in range(result_shape[1]):
#         for b in range(result_shape[2]):
#             for n in range(result_shape[3]):
#                 # d 차원에 대해 q와 k를 곱하고 합산합니다.
#                 temp_sum = tf.reduce_sum(q[i, b, n, :] * k[j, b, n, :], axis=-1)
#                 print(temp_sum)
#                 # 결과 텐서에 값을 할당합니다.
#                 result[i, j, b, n].assign(temp_sum)

# print(result)

tf.Tensor(199, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(117, shape=(), dtype=int32)
tf.Tensor(53, shape=(), dtype=int32)
tf.Tensor(58, shape=(), dtype=int32)
tf.Tensor(73, shape=(), dtype=int32)
tf.Tensor(61, shape=(), dtype=int32)
tf.Tensor(72, shape=(), dtype=int32)
tf.Tensor(91, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32)
tf.Tensor(120, shape=(), dtype=int32)
tf.Tensor(85, shape=(), dtype=int32)
<tf.Variable 'Variable:0' shape=(1, 2, 2, 3) dtype=int32, numpy=
array([[[[199,  16, 117],
         [ 53,  58,  73]],

        [[ 61,  72,  91],
         [ 17, 120,  85]]]], dtype=int32)>


In [ ]:
print(q[0,0,1,:])
print(k[0,0,1,:])
print(q[0,0,1,:]*k[0,0,1,:])


tf.Tensor([0 0 2 8], shape=(4,), dtype=int32)
tf.Tensor([9 8 0 2], shape=(4,), dtype=int32)
tf.Tensor([ 0  0  0 16], shape=(4,), dtype=int32)


In [ ]:
for i in range(1):
    print(i)

0


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# 주어진 텐서 값
tensor_values = np.array([[[[7, 9, 6, 6],
                            [0, 0, 2, 8],
                            [4, 6, 7, 4]],

                           [[0, 5, 7, 2],
                            [3, 6, 9, 1],
                            [2, 8, 3, 2]]]])

# 텐서의 모양
shape = tensor_values.shape

# 서브플롯을 생성합니다.
fig = make_subplots(
    rows=1, cols=shape[1],  # shape[1]는 2차원의 크기입니다.
    specs=[[{'type': 'scatter3d'} for _ in range(shape[1])]],
    subplot_titles=[f'Slice {i+1}' for i in range(shape[1])]
)

# 각 슬라이스를 시각화합니다.
for slice_index in range(shape[1]):
    # x, y, z 좌표를 생성합니다.
    x, y, z = np.indices((shape[2], shape[3], 1)).reshape(3, -1)
    # 현재 슬라이스의 값들
    values = tensor_values[0, slice_index, :, :].flatten()
    # 텍스트 라벨
    text_labels = [f'{value}' for value in values]

    # 산점도를 추가합니다.
    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers+text',
            marker=dict(
                size=5,
                color=values,                # 값에 따라 색상을 설정합니다.
                colorscale='Viridis',   # 색상 스케일
                opacity=0.8
            ),
            text=text_labels,                # 텍스트 라벨을 추가합니다.
            textposition='top center'       # 텍스트 위치 설정
        ),
        row=1, col=slice_index+1
    )

# 레이아웃을 업데이트합니다.
fig.update_layout(
    title='3D Tensor Visualization',
    height=600,
    width=1000
)

# 그래프를 표시합니다.
fig.show()


In [ ]:
import tensorflow as tf

# 주어진 배열을 이용하여 (2,2,2,2) 차원의 텐서를 만듭니다.
data = [[[[1, 1],
          [1, 1]],
         [[2, 2],
          [2, 2]]],
          
        [[[3, 3],
          [3, 3]],
         [[4, 4],
          [4, 4]]]]

# 위 데이터를 텐서플로우 상수로 변환합니다.
sentence = tf.constant(data, dtype=tf.int32)

# A의 형태를 확인하여 (2,2,2,2) 차원의 텐서가 맞는지 확인합니다.
print(A.shape)

(2, 2, 2, 2)


In [ ]:
resulta = tf.einsum("ibnd,jbnd->ijbn", sentence, sentence)
resulta

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int32, numpy=
array([[[[ 2,  2],
         [ 8,  8]],

        [[ 6,  6],
         [16, 16]]],


       [[[ 6,  6],
         [16, 16]],

        [[18, 18],
         [32, 32]]]], dtype=int32)>

In [ ]:
print(sentence[0,1,1,:])
print(sentence[0,1,1,:])

tf.Tensor([2 2], shape=(2,), dtype=int32)
tf.Tensor([2 2], shape=(2,), dtype=int32)


In [ ]:



# 결과를 저장할 리스트를 초기화합니다.
result_list = []

# 직접 계산하는 방식
for i in range(sentence.shape[0]):
    for j in range(sentence.shape[0]):  # 범위를 20으로 변경합니다.
        for b in range(sentence.shape[1]):
            for n in range(sentence.shape[2]):
                # temp_sum을 계산합니다.
                temp_sum = tf.reduce_sum(sentence[i, b, n, :] * sentence[j, b, n, :], axis=-1)
                # 결과 리스트에 추가합니다.
                result_list.append(temp_sum.numpy())

# 리스트를 텐서로 변환합니다.
result_tensor = tf.convert_to_tensor(result_list, dtype=tf.int32)
result_tensor = tf.reshape(result_tensor, (2, 2, 2, 2))

# einsum을 사용하는 방식
einsum_result = tf.einsum("ibnd,jbnd->ijbn", sentence, sentence)

# 두 결과가 동일한지 검증합니다.
print(tf.reduce_all(result_tensor == einsum_result).numpy())
print('result_tensor',result_tensor)

True
result_tensor tf.Tensor(
[[[[ 2  2]
   [ 8  8]]

  [[ 6  6]
   [16 16]]]


 [[[ 6  6]
   [16 16]]

  [[18 18]
   [32 32]]]], shape=(2, 2, 2, 2), dtype=int32)


In [ ]:
A = tf.reshape(embedded_sentence_tensor,(2,2,2,2))

In [ ]:
A

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int32, numpy=
array([[[[1, 1],
         [1, 1]],

        [[2, 2],
         [2, 2]]],


       [[[3, 3],
         [3, 3]],

        [[4, 4],
         [4, 4]]]], dtype=int32)>